In [1]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

def ipex_jit_test():
    model = models.resnet18()
    inputs = torch.randn(5, 3, 224, 224)
    model.eval()
    model = torch.jit.trace(model, torch.randn(5, 3, 224, 224))
    model = torch.jit.freeze(model)

    def foo(inputs, p=None):
        for i in range(30):
            model(inputs)

    with torch.profiler.profile(
                activities=[torch.profiler.ProfilerActivity.CPU],
                record_shapes=True,
            ) as p:
        foo(inputs, p=p)
    print(p.key_averages().table(sort_by="cpu_time_total", row_limit=10))

ipex_jit_test()
print("=======================================\n")
import intel_extension_for_pytorch as ipex
ipex_jit_test()

/home/taosy/repo/pytorch/vision/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
STAGE:2023-08-14 21:10:59 2340048:2340048 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-08-14 21:11:00 2340048:2340048 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-08-14 21:11:00 2340048:2340048 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                          forward         0.85%       6.027ms        99.53%     703.956ms      23.465ms            30  
               aten::_convolution         1.15%       8.145ms        84.74%     599.341ms     998.902us           600  
         aten::mkldnn_convolution        82.73%     585.091ms        83.59%     591.196ms     985.327us           600  
                 aten::max_pool2d         0.05%     321.000us         6.45%      45.599ms       1.520ms            30  
    aten::max_pool2d_with_indices         6.40%      45.278ms         6.40%      45.278ms       1.509ms            30  
                      aten::relu_       

[W OperatorEntry.cpp:153] Warning: Warning only once for all operators,  other operators may also be overrided.
  Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::conv1d(Tensor input, Tensor weight, Tensor? bias=None, int[1] stride=[1], SymInt[1] padding=[0], int[1] dilation=[1], int groups=1) -> Tensor
    registered at /opt/conda/conda-bld/pytorch_1686640406433/work/build/aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: AutocastCPU
  previous kernel: registered at /opt/conda/conda-bld/pytorch_1686640406433/work/aten/src/ATen/autocast_mode.cpp:216
       new kernel: registered at /home/taosy/repo/intel/frameworks.ai.pytorch.ipex-cpu/csrc/cpu/autocast/autocast_mode.cpp:164 (function operator())
STAGE:2023-08-14 21:11:02 2340048:2340048 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-08-14 21:11:02 2340048:2340048 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-08-14 21:11:02 

---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            forward         1.21%       8.161ms        99.17%     667.386ms      22.246ms            30  
             ipex_prepack::convolution_add_relu_run        50.90%     342.533ms        50.90%     342.533ms       1.476ms           232  
                 ipex_prepack::convolution_relu_run        33.13%     222.940ms        33.24%     223.707ms     857.115us           261  
                                     dil_max_pool2d         4.79%      32.256ms         4.88%      32.849ms       1.133ms            29  
                      ipex_prepack